In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
keras = tf.keras
imgX = 244
imgY = 244
imgZ = 1
EPOCHS = 10

In [5]:
classes = os.listdir('./assets')
# classes = classes[:2]
classes

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [ ]:
def read_img(path):
    img = cv2.imread(path)
    img = img / 255
    # print(img.shape)
    img.resize(imgX,imgY,imgZ)
    return img

In [ ]:
# get training data from all classes into one
images = []
labels = []
for i, c in enumerate(classes):
    n = 50
    for f in os.listdir(f'./assets/{c}'):
        img = read_img(f'./assets/{c}/{f}')
        images.append(img)
        labels.append(i)
        n -= 1
        if n < 0:
            break
print(set(labels))

In [ ]:
def encode_labels(labels):
    hashlabels = []
    for x in labels:
        hashlabels.append([1 if i == x else 0 for i in range(len(classes))])
    return hashlabels
labels = encode_labels(labels)
labels[0]

In [ ]:
print(f'Total images: {len(images)}\nTotal labels: {len(labels)}\nImage Size: {images[0].shape}')

In [ ]:
# split in train and test
X_train, X_test = images[:int(len(images)*0.8)], images[int(len(images)*0.8):]
y_train, y_test = labels[:int(len(labels)*0.8)], labels[int(len(labels)*0.8):]
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(imgX,imgY,imgZ)))
model.add(keras.layers.Dense(sum([len(classes), imgX*imgY*imgZ])/2, activation=tf.nn.relu))
model.add(keras.layers.Dense(len(classes), activation='softmax'))
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train, y_train,epochs=EPOCHS)
model.save('flower_model.h5')

In [ ]:
# model = keras.models.load_model('flower_model.h5')
loss, accuracy = model.evaluate(X_test, y_test)
loss, accuracy

In [ ]:
def predict_image(path):
    img = read_img(path)
    img = np.array([img])
    x = model.predict(img)
    return classes[np.argmax(x[0])]

In [ ]:
predict_image('./assets/dandelion/11124381625_24b17662bd_n.jpg')